In [ ]:
!pip install camelot-py
!pip install opencv-python
!pip install tabula-py==2.9.0
!pip install langchain
!pip install langchain-core==0.1.40
!pip install jpype1

In [22]:
import glob
import json
import os
import re
import time
import warnings
from datetime import datetime
from pathlib import Path
from urllib.parse import urlparse

import camelot
import numpy as np
import pandas as pd
import requests
import tabula
import yaml

### Helper functions

In [4]:
def extract_text_from_file(df, file_path):
    file_name = file_path.split("/")[-1]
    file_type = file_name.split(".")[-1]
    if file_type == "pdf":
        loader = PyPDFLoader(file_path)
    elif file_type == "docx":
        loader = Docx2txtLoader(file_path)
    else:
        return df

    pages = loader.load_and_split()
    for page_number, page in enumerate(pages, start=1):
        # Each page's text is added as a new row in the DataFrame
        new_row = pd.DataFrame(
            {
                "file": [file_name],
                "page_number": [page_number],
                "text": [page.page_content],
            }
        )
        df = pd.concat([df, new_row], ignore_index=True)

    return df


def extract_tables(file_path, pages="all", package="tabula"):
    if package == "camelot":
        # Extract tables with camelot
        # flavor could be 'stream' or 'lattice', for documents where tables do not have clear borders, the stream flavor is generally more appropriate.
        tables = camelot.read_pdf(file_path, pages=pages, flavor="stream")
    else:
        tables = tabula.read_pdf(file_path, pages=pages, stream=True, silent=True)

    # Convert tables to JSON
    tables_json = []
    for idx, table in enumerate(tables):
        if package == "camelot":
            page_number = table.parsing_report["page"]
            data = table.df.to_json(orient="records")
        else:
            page_number = ""
            data = table.to_json(orient="records")

        data = {
            "table_number": idx,
            "page_number": page_number,
            "data": data,
        }
        tables_json.append(data)
    return tables_json

In [19]:
folder_path = "../data/raw"
pathlist = Path(folder_path).glob("*.pdf")
filenames = []
for file_path in pathlist:
    filename = os.path.basename(file_path)
    filenames.append(filename)
len(filenames)

1

In [18]:
df = pd.DataFrame()
for filename in filenames:
    file_path = folder_path + "/" + filename
    file_name = os.path.basename(file_path)
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} process {file_name}")
    # Initialize an empty df
    df_file = pd.DataFrame(columns=["file", "text"])
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} extract text")
    try:
        df_file = extract_text_from_file(df_file, file_path)
    except Exception as e:
        print("----Error: cannot extract text")
        print(f"----error: {e}")
    df = pd.concat([df, df_file])

df.head()

Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)


2024-04-28 00:55:29 process GPTsareGPTs.pdf
2024-04-28 00:55:29 extract text


,file,text,page_number
0,GPTsareGPTs.pdf,WORKING PAPER\nGPTs are GPTs: An Early Look at...,1.0
1,GPTsareGPTs.pdf,WORKING PAPER\nFigure 1: Taken directly from G...,2.0
2,GPTsareGPTs.pdf,WORKING PAPER\nlevels in GPT-4 responses and b...,3.0
3,GPTsareGPTs.pdf,technology. Our evidence supports a wider impa...,4.0
4,GPTsareGPTs.pdf,WORKING PAPER\npolicymakers to predict and reg...,5.0


In [ ]:
for filename in filenames:
    file_path = folder_path + "/" + filename
    file_name = os.path.basename(file_path)
    df_file = pd.DataFrame()
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} process {file_name}")

    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} extract table")

    all_tables = []
    for package in ["camelot", "tabula"]:
        print(
            f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} extract table with {package}"
        )
        try:
            tables_from_package = extract_tables(
                file_path, pages="all", package=package
            )  # list of json
            for table in tables_from_package:
                all_tables.append({"table": table, "source": package})
        except Exception as e:
            print("----Error: cannot extract table")
            print(f"----error: {e}")

# Now you can access each table along with its source
for entry in all_tables:
    print(f"Source: {entry['source']}, Table: {entry['table']}")

In [126]:
test_case = list(filter(lambda element: element["source"] == "camelot", all_tables))[
    23
]["table"]["data"]
json_data = json.loads(test_case)
df = pd.DataFrame(json_data, columns=None)
df.drop(df.index[0])

,0,1,2,3,4,5,6,7,8,9,10,11
1,Job,Preparation,Education,Example Occupations,Median,Tot Emp,HUUU,MUUU,HVVV,MVVV,HZZZ,MZZZ
2,Zone,Required,Required,,Income,(000s),,,,,,
3,1,None or little,High school,"Food preparation workers,","$30,230","13,100",0.03,0.04,0.06,0.06,0.09,0.08
4,,(0-3 months),diploma or GED,"dishwashers, ﬂoor sanders",,,,,,,,
5,,,(otional),,,,,,,,,
6,2,Some (3-12,High school,"Orderlies, customer","$38,215","73,962",0.07,0.12,0.16,0.20,0.24,0.27
7,,months),diploma,"service representatives,",,,,,,,,
8,,,,tellers,,,,,,,,
9,3,Medium (1-2,"Vocational school,","Electricians, barbers,","$54,815","37,881",0.11,0.14,0.26,0.32,0.41,0.51
10,,years),"on-the-job training,",medical assistants,,,,,,,,
